<a href="https://colab.research.google.com/github/PyMap/AUPY/blob/master/Modulo%203/03_3_clases_objetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grandes volúmenes de datos



## [Properati](https://blog.properati.com.ar/category/contenidos-data/), un ejemplo aplicado al mundo del real estate ... 

Como científico de datos urbanos, una dimensión de interés es lo que sucede en el mundo de la vivienda. Al menos aquella que circula dentro de un mercado formal.

Construir nuestros propios índices de precios y seguirlos a lo largo de un período, evaluar la distribución espacial de las distintas tipologías o incluso estudiar los niveles de inflación. Todas, preguntas que podríamos llegar a hacernos frente a un análisis que involucre a la viviendas como nuestro objeto de estudio.

La noción de BigData puede ser a veces un poco difusa. Sin embargo, no queda duda que este concepto está innegablemente ligado a la noción de tiempo. Más datos suceden en más tiempo. Por eso, aprender a trabajar con conjuntos de datos administrados, accesibles a lo largo del tiempo es tan importante. Porque nos abre las puertas para trabajar con cuestiones de interés.

El portal inmobiliario `properati` 🏢 es un buen caso de uso para hablar de Big Data y ciencia de datos espaciales. Una fuente que se actualiza constantemente y que nos permitiría hasta incluso rutinizar tareas que se repitan periódicamente.

`properati`<img src='https://drive.google.com/uc?id=1I_RcTUGu4xk5Sh38v-QFINPmGOYgum4-' width=25px, height=25/>

### El conjunto de datos 

Properati almacena sus registros de publicaciones en distintos conjuntos de datos. Todos ellos correspondientes a múltiples países de latinoamérica (algo también interesante, no?). Imagínense todas las preguntas que podríamos responder, o rutinas que podríamos estandarizar para construir nuestra propia información.

Para esta clase, vamos a trabajar con uno de los extractos descargable en formato csv. L@s que estén interesados en ver cómo funcionan las consultas en la consola de google cloud utilizando Big Query puede ir al material complementario. Como decíamos, estos extractos son porciones organizadas en fechas que se actualizan diariamente. Con el que trabajaremos corresponde a argentina. 

Cada tabla correspondiente al conjunto de datos de cada país tiene el siguiente esquema:


* **ad_type** - Tipo de aviso (Propiedad, Desarrollo).
* **id** - Identificador del aviso. No es único: si el aviso es actualizado por la inmobiliaria (nueva versión del aviso) se crea un nuevo registro con la misma id pero distintas fechas.
* **start_date** - Fecha de alta del aviso.
* **end_date** - Fecha de baja del aviso.
* **created_on** - Fecha de alta de la primera versión del aviso.
* **place** - Campos referidos a la ubicación de la propiedad o del desarrollo.
  * **lat** - Latitud.
  * **lon** - Longitud.
  * **l1** - Nivel geopolítico 1: país.
  * **l2** - Nivel geopolítico 2: usualmente provincia.
  * **l3** - Nivel geopolítico 3: usualmente ciudad/localidad.
  * **l4 a l6** - Niveles geopolíticos 4 a 6: usualmente barrio.
* **property** - Campos relativos a la propiedad (vacío si el aviso es de un desarrollo).
  * **operation_type** - Tipo de operación (Venta, Alquiler y Alquiler temporal ).
  * **property_type** - Tipo de propiedad (Casa, Departamento, PH, etc.).
  * **rooms** - Cantidad de ambientes (útil en Argentina).
  * **bedrooms** - Cantidad de dormitorios (útil en el resto de los países).
  * **bathrooms** - Cantidad de baños.
  * **surface_total** - Superficie total en m².
  * **surface_covered** - Superficie cubierta en m².
  * **price** - Precio publicado en el anuncio.
  * **currency** - Moneda del precio publicado.
  * **price_period** - Periodo del precio (Diario, Semanal, Mensual)
  * **title** - Título del anuncio.
  * **description** - Descripción del anuncio.


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/python_mdg/data_cursos/ar_properties.csv.gz')

In [ ]:
df.columns

## Fechas, días y el uso de `datetime`

**Datetime** es un modulo de pandas diseñado para manipular fechas y horas. Las clases que provee permite tanto operaciones aritméticas como la extraccion de campos como dia, mes, fecha y hora de una manera rapida y eficiente.

In [ ]:
# importamos el modulo
from datetime import datetime

In [ ]:
df.start_date[0]

In [ ]:
# convirtamos el primer registro de la serie Fecha a dias. Esto nos devolvera un valor posicional
datetime.strptime(df.start_date[0],'%Y-%m-%d').weekday()

In [ ]:
 # Y obtengamos el dia de la semana para todo el dataset
dia_semana = df.start_date.apply(lambda x: datetime.strptime(x,'%Y-%m-%d').weekday())

In [ ]:
dia_semana

In [ ]:
df['dia_semana'] = dia_semana.copy()

In [ ]:
# cambiamos de posicional a nombres
df['dia_semana'].replace({0:'lunes', 
                          1:'martes', 
                          2:'miercoles', 
                          3:'jueves', 
                          4:'viernes', 
                          5:'sabado', 
                          6:'domingo'}, inplace=True)

In [ ]:
df['dia_semana'].value_counts()

Otro camino que podriamos haber tomado para trabajar con datetime, es a partir del metodo to_datetime de pandas. Veamos algunos ejemplos...

In [ ]:
# esto nos devuelve el mes
pd.to_datetime(df.start_date).dt.month

In [ ]:
# la fecha en formato datetim64
pd.to_datetime(df.start_date).dt.date.astype('datetime64') 

In [ ]:
# el dia de la semana
pd.to_datetime(df.start_date).dt.weekday

... volviendo a lo nuestro, nosotros habíamos generado una nueva Serie con el día de la publicación. Esto nos permitiría hablar de algo puntual. Si hay algún día de la semana en el que "se publique más".

Tomemos como excusa esta consigna para introducir algunos conceptos generales de ploteo.

In [ ]:
df['dia_semana'].value_counts().plot(kind='bar',grid=True, color='#2F9599');

In [ ]:
# le asignamos un nuevo label al axis 0
orden = ["lunes", "martes", "miercoles", "jueves", "viernes", "sabado","domingo"]
df['dia_semana'].value_counts().reindex(orden)

In [ ]:
# y volvemos a plotear
df['dia_semana'].value_counts().reindex(orden).plot(kind='bar',grid=True, color='#2F9599');

In [ ]:
# mes y año de inicio
df['start_mes'] = pd.to_datetime(df.start_date).dt.month
df['start_year'] = pd.to_datetime(df.start_date).dt.year

In [ ]:
# vemos los años involucrados en la serie
df['start_year'].unique()

In [ ]:
# nuestro indice podria ser anual. De marzo ...
df.loc[df['start_year']==2020].start_mes.unique()

In [ ]:
# ... a marzo
df.loc[df['start_year']==2021].start_mes.unique()

In [ ]:
'{}-{}'.format(df['start_year'][0],df['start_mes'][0])

In [ ]:
# creamos una funcion para armar una etiqueta con el periodo mes/año
def etiqueta_aviso(x):
    fecha = '{}-{}'.format(x['start_year'],x['start_mes'])
    return fecha

Fíjense cómo aplicamos nuestra función regular. Sí! por medio de una función anónima.  Esta vez, el apply se aplica a todo el df. Por eso, esta vez, el place holder (x) no refiere a una serie o columna. La `x` ahora, es el dataframe. Por eso, nuestra función regular toma nombres de columnas. De este modo, es que también resulta necesario definir en qué sentido de los ejes será recorrido el dataframe. Nosotros ponemos 1, porque queremos ir recorriendo las filas de más de una columna a la vez.

In [ ]:
# la aplicamos
df['etiqueta_fecha'] = df.apply(lambda x: etiqueta_aviso(x), axis=1)

In [ ]:
df['etiqueta_fecha']

In [ ]:
# vemos cuales son los tipos de operación posibles
df.operation_type.unique()

In [ ]:
# nos quedamos con los alquileres
alquileres = df.loc[df['operation_type']=='Alquiler']

### Construyendo series con tablas pivote 

> **pivot tables**

Las tablas pivot suelen ser muy utiles para sumarizar informacion. Mas alla de la implementacion de pandas, este es un recurso que atraviesa distintas herramientas.

Lo que hace esto, es basicamente transformar la forma (shape) de nuestro dataframe a partir de un valor que buscamos agregar. El metodo pivot_table permite especificar dicho valor y definir la funcion de agregacion (aggfunc) con la que lo vamos a sumarizar.

Veamos a continuacion que, para utilizar este metodo, pasamos un dataframe y definimos los values a ser sumarizados, los index y columns de nuestro nuevo marco de datos - pueden refrescar la digresion que hicimos en la clase anterior sobre axis - y la aggfunc. 

In [ ]:
df.columns

Exploremos nuestra variable de precios

In [ ]:
# ya vemos algo raro
df['price'].max()

In [ ]:
# efectivamente, nunca filtramos nuestra base por geografía. Empecemos por argentina...
df['currency'].unique()

In [ ]:
# nos quedamos con pesos argentinos
alquileres_arg = alquileres.loc[alquileres['currency']=='ARS'].copy()

In [ ]:
# la media ya tiene un valor mas razonable, prosigamos
alquileres_arg.price.mean()

In [ ]:
# creamos una tabla pivot
pivot_ = pd.pivot_table(alquileres_arg,
                        values="price",
                        index=alquileres_arg['etiqueta_fecha'], # se puede hacer en sentido del eje 0
                        #columns = aliquileres_arg['etiqueta_fecha'] # o del eje 1
                        aggfunc = 'mean',fill_value=0)

In [ ]:
pivot_.head(2)

In [ ]:
# veo si los puedo ordenar de menor a mayor
pivot_.sort_index(ascending=True).index

In [ ]:
# acomodamos un poco a mano
periodos_ordenados = ['2020-3', '2020-4', '2020-5', 
                      '2020-6','2020-7', '2020-8', '2020-9', 
                      '2020-10', '2020-11', '2020-12',
                      '2021-1', '2021-2', '2021-3']

In [ ]:
# reindexamos
pivot_.reindex(periodos_ordenados)

In [ ]:
# y volvemos a plotear
pivot_.reindex(periodos_ordenados).plot(kind='line', grid=True, color='#FF6347');

Bueno, bien. Logramos construir una primera serie y plotearla. Tratemos de apuntar ahora a algo con un poco más de sentido conceptual. Este promedio que acabamos de construir no sólo no tiene en cuenta los efectos de la inflación sino también los tipos de productos considerados.

Tratemos de ser un poco más específicos. Determinemos alguna región, tipo de producto y, para terminar, intentemos deflacionar nuestra serie.

### Deflactando series

Para deflactar una serie de valores nominales, es necesario contar con alguna referencia. La traducción de valores nominales a reales (o corrientes y constantes), contempla el uso de un índice que nos permita evaluar cómo se comportaron los precios (o algunos) de la economía en relacion al poder adquisitivo. Para así, mitigar el impacto de la inflación en el bien que estamos estudiando. 

Por decirlo de alguna manera, vamos a tratar de expresar lo que vale un precio de un momento n+1 considerando a n como momento inicial. Cuánto vale algo en pesos de marzo 2021, por ejemplo.  

Lo primero con lo que vamos a trabajar es con el [IPC-Indec](https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-31). 

Vamos a seleccionar la Región GBA y a buscar alguna apertura que nos interese para seguir estudiando los precios de los alquileres. Seleccionemos primero nuestra región.

In [ ]:
# disponemos de los siguientes niveles administrativos
alquileres_arg.l2.unique()

In [ ]:
# nos quedamos con la Region GBA
jurisdicciones = ['Bs.As. G.B.A. Zona Oeste', 'Capital Federal',
                  'Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Sur']
alquileres_gba = alquileres_arg[alquileres_arg.l2.isin(jurisdicciones)]

In [ ]:
# chequeamos el tipo de bien que vamos a estudiar
alquileres_gba.property_type.unique()

In [ ]:
# quedemonos solamente con los departamentos
alquileres_gba = alquileres_gba[alquileres_gba.property_type.isin(['Departamento'])].copy()

#### El IPC - Indec

Listo, ya tenemos una serie de precios más depurada. Ahora trabajemos en acomodar nuestro índice de precios. El mismo está decargado en el el directorio de data del curso. Vemos cómo se ve...

In [ ]:
!pip install xlrd

In [ ]:
ipc_crudo = pd.read_excel('https://storage.googleapis.com/python_mdg/data_cursos/sh_ipc_aperturas.xls',
                           sheet_name='Variación mensual aperturas', header=None)

In [ ]:
# hacemos slicing desde las filas 5 a la 53, el IPC para GBA
ipc_crudo = ipc_crudo.iloc[5:53].copy()

In [ ]:
# renombramos columnas
ipc_crudo.columns = ipc_crudo.iloc[0]

In [ ]:
ipc_indec = ipc_crudo.iloc[3:].copy()

In [ ]:
ipc_indec.columns

In [ ]:
# listemos los metodos/atributos de la clase datetime
dir(datetime)

In [ ]:
# armo de nuevo el nombre de las columnas
new_columns = []
for i in ipc_indec.columns:
    if i != 'Región GBA':
        y = i.year # el item que estamos iterando ya es un datetime!
        m = i.month
        fecha = '{}-{}'.format(y,m)
        new_columns.append(fecha)
    else:
        new_columns.append(i)

In [ ]:
ipc_indec.columns = new_columns

In [ ]:
# reseteamos nuestro indice
ipc_indec.reset_index(inplace=True)

In [ ]:
# y borramos lo que era el previo
ipc_indec.drop(columns='index', inplace=True)

Ya tenemos nuestro IPC con las siguientes especificaciones. 

```
Índice de precios al consumidor con cobertura nacional. Resultados por región (GBA)

Período de referencia: Diciembre 2016=100

Variaciones mensuales de enero de 2017 a abril de 2021. Resultados según principales aperturas de la canasta
```

In [ ]:
ipc_indec.head()

In [ ]:
def aperturas_ipc_indec(df, rubro):
    aperturas = df.set_index('Región GBA')
    apertura_elegida = aperturas.loc[rubro]
    return apertura_elegida

In [ ]:
# el IPC para cada periodo y apertura
aperturas_ipc_indec(ipc_indec, 'Alquiler de la vivienda')

Una vez que tenemos nuestro IPC limpio, listo para usar, no tenemos que perder de vista que el mismo va a ser aplicado a otro dataframe (nuestra tabla pivot con promedios nominales por período). 
Nosotros vamos a tomar el precio promedio para las operaciones de alquiler durante los últimos 12 meses desde marzo del corriente año. Es decir, de marzo de 2020 a marzo de 2021. 

In [ ]:
# creamos una funcion para ajustar los pesos nominales por IPC
def deflactar_serie(x,y,z,ipc,rubro,ipc_indec=True):
    '''
    x (str): periodo  d
    '''
    # guardamos el periodo y su precio nominal
    periodo = x
    precio_nominal = y
    periodo_base = z

    if ipc_indec:
        # devuelve una serie con la variacion del rubro 
        serie = aperturas_ipc_indec(ipc, rubro)
    else:
        serie = ipc
    # seleccionamos el valor del índice para una apertura y periodo determinado
    ipc_periodo = serie.loc[periodo] 
    ipc_base = serie.loc[periodo_base]  
    coeficiente = ipc_periodo/ipc_base
    precio_constante = precio_nominal / coeficiente
    
    return periodo, precio_nominal, coeficiente, ipc_base, ipc_periodo, precio_constante


Preparemos ahora el dataset de alquileres, tenemos que quedarnos con sus periodos y el precio promedio para los casos del GBA.

In [ ]:
# creamos nuestros promedios de precio por periodo para GBA
pivot_gba = pd.pivot_table(alquileres_gba,
                           values="price",
                           index=alquileres_gba['etiqueta_fecha'],
                           aggfunc = 'mean',fill_value=0)

In [ ]:
pivot_gba.index

In [ ]:
# ajustamos un poco
periodos_ordenados = ['2020-3', '2020-4', '2020-5', '2020-6', 
                      '2020-7', '2020-8', '2020-9', '2020-10', 
                      '2020-11', '2020-12','2021-1', '2021-2', '2021-3']
gba_ordenado = pivot_gba.reindex(periodos_ordenados)

In [ ]:
# y reseteamos el index
gba_ordenado.reset_index(inplace=True)

In [ ]:
# renombamos columnas
gba_ordenado.columns = ['period', 'price']

In [ ]:
gba_ordenado.set_index('period').loc['2020-3']

In [ ]:
ipc_indec

In [ ]:
# almacenamos nuestro resultado en una serie de tuplas
resultado = gba_ordenado.apply(lambda x: deflactar_serie(x.period, 
                                                         x.price,
                                                         '2020-3', # mes base 
                                                         ipc_indec, 
                                                         'Nivel general'), 
                               axis=1)

In [ ]:
resultado

In [ ]:
# guardamos el resultado
serie_ajustada = resultado.apply(pd.Series)

# renombramos las columnas
columnas = ['periodo', 'precio_nom', 'coeficiente', 'ipc_base', 'ipc_per', 'precio_con']
serie_ajustada.columns = columnas

In [ ]:
serie_ajustada

In [ ]:
from matplotlib import pyplot as plt # importan la clase pyplot de matplotlib

In [ ]:
# ahora, creemos una figura (este tema lo van a retomar en la clase de visualizacion!)
fig, ax = plt.subplots()
serie_ajustada.iloc[1:].set_index('periodo')['precio_nom'].plot(kind='line', grid=True, color='r')
serie_ajustada.iloc[1:].set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='b');

La evolución del precio promedio para departamentos en alquiler de la Región GBA. En azul, los precios en base a marzo de 2020 (pesos de ese mes/año). En azul, los precios nominales.

In [ ]:
# creamos algunas series para ver cuánto variaron de año a año en términos porcentuales
serie_ajustada['pct_nom']=serie_ajustada['precio_nom'].pct_change() * 100
serie_ajustada['pct_con']=serie_ajustada['precio_con'].pct_change() * 100
serie_ajustada['pct_ipc']=serie_ajustada['ipc_per'].pct_change() * 100

In [ ]:
# y ploteamos!
fig, ax = plt.subplots()
serie_ajustada.iloc[0:].set_index('periodo')['pct_nom'].plot(kind='line', grid=True, color='r',ax=ax)
serie_ajustada.iloc[0:].set_index('periodo')['pct_con'].plot(kind='line', grid=True, color='b', ax=ax);

Vemos algunas cosas acá. Ahora bien, es correcto ajustar asi?, considerando sólo el coportamiento de los precios y no de los ingresos? sobretodo si vemos cómo funciona nuestro índice a lo largo del tiempo ...

In [ ]:
serie_ajustada.iloc[0:].set_index('periodo')['pct_ipc'].plot(kind='line', grid=True, color='g');

#### El ICL - BCRA

Ahora bien, el índice que estamos usando solamente contempla el impacto de los precios (la variacion de los bienes ofertados). Como que nos falta algo para poder ajustar y que el grafico tenga mas sentido.

Para tener un análisis un poco más completo, deberíamos incluir de alguna manera cómo evolucionó también el poder de compra o pago de los salarios. 

Un muy buen atajo podría ser, en lugar construir una serie de ingresos y combinar con el IPC, utilizar el ICL. El índice de Contratos de Locación.

[Desde agosto de 2020, con la sanción de la Nueva Ley de Alquileres](https://www.argentina.gob.ar/noticias/el-bcra-oficializo-el-metodo-para-calcular-el-ajuste-anual-de-los-alquileres), los contratos de alquiler de los inmuebles destinados a uso habitacional se ajustan anualmente. El indice para hacerlo es el ICL, que podemos indagar en la web del [BCRA](http://bcra.gob.ar/PublicacionesEstadisticas/Principales_variables.asp)

In [ ]:
icl_bcra = pd.read_excel('https://storage.googleapis.com/python_mdg/data_cursos/indice_contratos_locacion.xls')

In [ ]:
icl_bcra['Mes'] = icl_bcra.Fecha.apply(lambda x: x.split('/')[2] + '-' + x.split('/')[1]) 

In [ ]:
# reemplazamos las comas y convertimos nuestro valores en float
icl_bcra['Valor'] = icl_bcra['Valor'].str.replace(',', '.', regex=False)
icl_bcra['Valor'] = icl_bcra.Valor.apply(lambda x: float(x))

In [ ]:
icl_bcra = icl_bcra.drop_duplicates(subset='Mes', keep='last')

In [ ]:
icl_bcra = icl_bcra[['Valor','Mes']].set_index('Mes')

In [ ]:
# renombramos nuestro indice, para que matchee con nuestra serie de valores
icl_bcra.index = icl_bcra.index.str.replace('-0', '-', regex=False)

In [ ]:
# asi tenemos que pasar el indice
icl_bcra['Valor']

In [ ]:
# como nuestro indice empieza en julio, recortamos nuestra serie nominal desde ahi
gba_ordenado_ = gba_ordenado.set_index('period').loc['2020-7':].reset_index()
resultado = gba_ordenado_.apply(lambda x: deflactar_serie(x.period, 
                                                         x.price,
                                                         '2020-7', # mes base 
                                                         icl_bcra.Valor, 
                                                         'Nivel general',
                                                         ipc_indec=False), 
                                axis=1)

In [ ]:
# hacemos las mismas transformaciones que hicimos antes para acomodar las tuplas
serie_ajustada = resultado.apply(pd.Series)
columnas = ['periodo', 'precio_nom', 'coeficiente', 'ipc_base', 'ipc_per', 'precio_con']
serie_ajustada.columns = columnas

In [ ]:
# el idice es mucho menos brusco que el anterior
serie_ajustada

In [ ]:
fig, ax = plt.subplots()
serie_ajustada.set_index('periodo')['precio_nom'].plot(kind='line', grid=True, color='r');
serie_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='b');

Ahora si, nuestros precios deflactados. Se puede apreciar que la serie ajustada por ICL es más suave que la nominal.

In [ ]:
serie_ajustada['pct_nom']=serie_ajustada['precio_nom'].pct_change() * 100
serie_ajustada['pct_con']=serie_ajustada['precio_con'].pct_change() * 100
serie_ajustada['pct_ipc']=serie_ajustada['ipc_per'].pct_change() * 100

In [ ]:
fig, ax = plt.subplots()
serie_ajustada.set_index('periodo')['pct_nom'].plot(kind='line', grid=True, color='r',ax=ax)
serie_ajustada.set_index('periodo')['pct_con'].plot(kind='line', grid=True, color='b', ax=ax)
serie_ajustada.set_index('periodo')['pct_ipc'].plot(kind='line', grid=True, color='g', ax=ax);

Confirmamos lo anterior, el cambio porcentual de los precios constantes fue un poco más bajo que el nominal. En verde, se aprecia cómo el ICL es mucho más suave que el IPC.

## Ejercicio sugerido (con solución)

> 1. **Armar con una tabla pivot el precio promedio por período para las distintas jurisdicciones que contemplamos para GBA.**

In [ ]:
# con una tabla pivot podemos obtener los promedios para todas las jurisdicciones
alquileres_gba_region = pd.pivot_table(alquileres_gba,
                                       values="price",
                                       index=alquileres_gba['etiqueta_fecha'],
                                       columns=alquileres_gba['l2'], # la jurisdiccion! 
                                       aggfunc = 'mean',fill_value=0)

In [ ]:
# lo pueden ver aca
alquileres_gba_region.head()

Entonces, algo que podríamos hacer es crear una función para seleccionar regiones de a una, y de paso, acomodar el dataframe a como sabemos que lo vamos a necesitar.

In [ ]:
# nuestra función para seleccionar regiones
def seleccionar_region(region):
    
    '''
    Selecciona una columna, siendo esta la region y sus precios
    en el df "alquileres_gba_region".
    '''
    periodos_ordenados = ['2020-3', '2020-4', '2020-5', '2020-6', 
                          '2020-7', '2020-8', '2020-9', '2020-10', 
                          '2020-11', '2020-12','2021-1', '2021-2', '2021-3']
    gba_regiones = alquileres_gba_region.reindex(periodos_ordenados)
    gba_regiones.reset_index(inplace=True)

    gba_region = gba_regiones[['etiqueta_fecha',region]].copy()

    gba_region.columns = ['period', 'price']
    return gba_region

In [ ]:
# podríamos empezar trabajando con zona norte
seleccionar_region('Bs.As. G.B.A. Zona Norte')

In [ ]:
# instanciamos zona norte
zona_norte = seleccionar_region('Bs.As. G.B.A. Zona Norte')

> **2. Elijan una región y calculen precios constantes. Si lo hacen por el camino indec deberan conseguir una serie de ingresos (puede ser por EPH, u otro mecanismo) para terminar de construir un indice.**

Como vimos en clase con el ICL, para deflactar, además de una medida que nos hable de precios debemos tener en cuenta la evolución de los ingresos. Entonces:

*2.1. Si deflactaramos por `icl`, no haría falta hacer ningún proceso adicional.*

*2.2. Si quisieramos usar el `ipc_indec` que construimos para GBA, deberíamos incorporar también una serie de ingresos.*

Como `2.1.` no nos insume ningún otro esfuerzo mas que volver a correr el notebook y cambiar las regiones, vamos a ir por `2.2.`. Tanto para alargar la serie deflactada como para agregar recursos adicionales que les queden disponibles.


Hasta ahora, nuestro ipc nos dice las variaciones porcentuales de los distintos bienes de la economía mes a mes (tomando como referencia un periodo de base). Esto deberíamos combinarlo con una serie salarial y armar un índice o coeficiente único.

Para hacer esto podríamos trabajar tanto con valores salariales como con variaciones porcentuales. A continuación les dejo algunos links donde pueden buscar referencias para construir series de salario:

**a)** La remuneración imponible promedio de los trabajadores estables - [RIPTE](https://www.argentina.gob.ar/trabajo/seguridadsocial/ripte)

**b)** Ministerio de Trabajo - [estadísticas](http://www.trabajo.gob.ar/estadisticas/Bel/ingresos.asp). Este portal incluye varias alternativas, desde construcciones trimestrales por EPH/EAHU hasta negociaciones colectivas.

**c)** Subsecretaría de Programación Macroeconómica de la Nación. Muchos de sus datasets están subidos al portal de datos `datos.gob.ar`.

Yo voy a ir por la opción `c` y voy a trabajar con [este dataset](https://datos.gob.ar/dataset/sspm-indice-salarios-base-octubre-2016/archivo/sspm_149.1) que ya está subido al directorio `data` del curso.

Entonces lo que voy a hacer es tratar de combinar el índice de salarios (que nos indica sus variaciones porcentuales a lo largo de los meses) con el índice de precios.

Para eso, primero cargamos el dataset en cuestión.

In [ ]:
# indice de salarios
salarios = pd.read_csv('https://storage.googleapis.com/python_mdg/data_cursos/indice-salarios-periodicidad-mensual-base-octubre-2016.csv')

In [ ]:
salarios.head()

Ahora, lo que vamos a tener que hacer es ajustar nuestro deflactor considerando dos escenarios: si pasamos un deflactor o indice ya listo para deflactar, o si pasamos un indice de precios y salarios para construir un nuevo indice o coeficiente.

Empecemos por armar una función que nos formatee nuestra sere de índices salariales

In [ ]:
# creamos una función para formatear nuestra serie de salarios
def formatea_isa(isa, serie='indice_salarios'):
    
    '''
    Formatea dataframe con indice de precios
    ...
    isa(df): indice de salarios - ver formato en dataframe
    serie(str): default con el nombre de la serie a utilizar 
                Nota: podrian elegir otra columna, para ello deberían cambiar
                      el valor del default cuando ejecutan la función indicando
                      el nombre de otra de las columnas disponibles!

    Devuelve:
    pandas.Series: serie con indice de salarios emparejado al año base del IPC
    '''
    # hacemos slicing sobre los strings de fecha para quedarnos con el periodo
    isa['periodo'] = isa.indice_tiempo.apply(lambda x: x[:7])
    # filtramos desde el periodo de referencia (octubre de 2016)
    isa_f = isa.set_index('periodo').loc['2016-10':].copy()
    # como nuestro ipc esta en base dicimiebre 2016, vamos a llevar el isa al mismo periodo
    isa_f.drop(columns='indice_tiempo', inplace=True) 

    # filtramos nuestro isa en base al periodo disponible del ipc,
    # esto se podría parametrizar en función de esa disponibilidad. Pero dejemoslo así, simple.
    isa_rebase = isa_f.apply(lambda x: x/isa_f.loc['2016-12'], axis=1)
    # renombramos los periodos para ajustar el formato que traemos de antes
    isa_rebase.index = isa_rebase.index.str.replace('-0', '-', regex=False)
    # nuestro ipc va de enero 2017 en adelante, hacemos slicing desde ahi
    isa_rebase = isa_rebase.loc['2017-1':].copy()
    return isa_rebase[serie]

In [ ]:
# aca nuestro indice de salarios formateado
formatea_isa(salarios).head()

Ahora que tenemos un índice de salarios listo tenemos que incorporar las modificaciones que mencionamos más arriba. Hacer que la función deflactora pueda trabajar con índices ya listos como el ICL, o bien, con índices que tienen que ser construidos. Para ello, incoroporemos la posibilidad de crear un nuevo índice a partir de salarios y precios:

In [ ]:
# vamos a ajustar un poco el codigo de nuestro deflactor para ajustarnos a la consigna
def deflactar_serie(pe,pr,ba, # en primer lugar, fijense que renombramos varios parametros
                    construye_d=True,
                    ipc=ipc_indec,
                    isa=salarios,
                    rubro_ipc='Nivel general',
                    rubro_isa='indice_salarios',
                    d=None,):
    
    '''
    Deflacta una serie de valores nominales
    ...
    pe(str): nombre de la serie con la etiqueta del periodo nominal
    pr(str): nombre de la serie con valores nominales
    ba(str): nombre del periodo que vamos a tomar como base
    construye_d(bool): construir el deflactor en el cuerpo de la funcion (e.g:False)
                       Nota alumnos: este parametro es default, si no lo especifican
                       a la hora de ejecutar la funcion se va a tomar True como valor.
    ipc(serie): serie de pandas con variaciones porcentuales de precios
    isa(serie): serie de pandas con variaciones porcentuales de salarios
    rubro_ipc(str): nombre de la apertura del ipc. El default es 'Nivel general', pero
                    también se puede cambiar cuando se ejecuta la función.
    rubro_isa(str): nombre del tipo de indice salarial de referencia. El default también
                    se puede cambiar por otro nombre.
    d(str): serie de pandas con un índice deflactor.
    Devuelve:
    pd.Series: Serie de tuplas.
    '''
    periodo = pe
    precio_nominal = pr
    periodo_base = ba

    if construye_d: # si esto evalua a True, 
        precios = aperturas_ipc_indec(ipc, rubro_ipc)
        salarios = formatea_isa(isa, rubro_isa)
        serie = salarios/precios.mean() # entonces construimos nuestro deflactor,
                                        # en nuestro caso expresamos las variaciones mensuales
                                        # de los salarios en función del promedio de las
                                        # variaciones de precios
    else:
        serie = d # sino, usamos uno ya armado (como el ICL)

    # en esta seccion deflactamos
    serie_periodo = serie.loc[periodo] 
    serie_base = serie.loc[periodo_base]  
    coeficiente = serie_periodo/serie_base
    precio_constante = precio_nominal / coeficiente

    return periodo, precio_nominal, coeficiente, serie_base, serie_periodo, precio_constante

Antes de aplicar nuestra función, un comentario sobre su uso. Si en lugar de haber hecho:

```
return periodo, precio_nominal, coeficiente, ipc_base, ipc_periodo, precio_constante
```

hubiésemos hecho solamente:

```
return precio_nominal
```

... esto nos habría permitido aplicar nuestra función directamente al dataframe original. Si, aquel que descargamos de properati y tenía alrededor de un millón de registros. Eso hubiese servido para crear una columna o serie de precios constantes allí mismo. Así:

```
df['price_constant'] = df.apply(lambda x: ...)
```

Pero como nuestro objetivo es otro (queremos ver esa serie de tuplas con todos los indicadores que construimos), vamos a simplemente dejarlo como está y ver qué resultado obtenemos para zona norte:

In [ ]:
zona_norte

In [ ]:
res_zn = zona_norte.apply(lambda x: deflactar_serie(pe=x.period, 
                                           pr=x.price,
                                           ba='2020-3',  
                                           rubro_ipc='Alquiler de la vivienda'), 
                          axis=1)

zn_ajustada = res_zn.apply(pd.Series)
zn_columnas = ['periodo', 'precio_nom', 'coeficiente', 'ipc_base', 'ipc_per', 'precio_con']
zn_ajustada.columns = zn_columnas

In [ ]:
fig, ax = plt.subplots()
zn_ajustada.set_index('periodo')['precio_nom'].plot(kind='line', grid=True, color='r')
zn_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='b');

> **3. Comparen esa serie con la serie de precios constantes para todo GBA (la que calculamos antes del ejercicio) - también se puede hacer la comparación con otra jurisdicción!**

Listo, lo que acabamos de hacer nos permitiría también comparar el comportamiento de los alquileres con otras jurisdicciones, por ejemplo, zona sur. Calculemosla.

In [ ]:
zona_sur = seleccionar_region('Bs.As. G.B.A. Zona Sur')

res_zs = zona_sur.apply(lambda x: deflactar_serie(pe=x.period, 
                                                  pr=x.price,
                                                  ba='2020-3',  
                                                  rubro_ipc='Alquiler de la vivienda'), 
                        axis=1)

zs_ajustada = res_zs.apply(pd.Series)
zs_columnas = ['periodo', 'precio_nom', 'coeficiente', 'ipc_base', 'ipc_per', 'precio_con']
zs_ajustada.columns = zn_columnas

In [ ]:
# así quedarían los precios nominales y reales en zona sur
fig, ax = plt.subplots()
zs_ajustada.set_index('periodo')['precio_nom'].plot(kind='line', grid=True, color='r')
zs_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='b');

In [ ]:
# así los precios constantes en zona norte y zona sur!
fig, ax = plt.subplots()
zs_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='g')
zn_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='y');

In [ ]:
res_gba = gba_ordenado.apply(lambda x: deflactar_serie(pe=x.period, 
                                                       pr=x.price,
                                                       ba='2020-3',  
                                                       rubro_ipc='Alquiler de la vivienda'), 
                             axis=1)

gba_ajustada = res_gba.apply(pd.Series)
gba_columnas = ['periodo', 'precio_nom', 'coeficiente', 'ipc_base', 'ipc_per', 'precio_con']
gba_ajustada.columns = gba_columnas

In [ ]:
# nuestros precios nominales y reales en todo el GBA
fig, ax = plt.subplots()
gba_ajustada.set_index('periodo')['precio_nom'].plot(kind='line', grid=True, color='r')
gba_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='b');

Ahora sí, el último paso. Comparemos las series constantes de Zona Norte y Sur, contextualizando vs la región GBA.

In [ ]:
# los precios constantes en contexto!
fig, ax = plt.subplots()
zs_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='g')
zn_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='y')
gba_ajustada.set_index('periodo')['precio_con'].plot(kind='line', grid=True, color='b');